# [Learning Spark Second Edition](https://github.com/databricks/LearningSparkV2)


### [Chapter 9](https://learning.oreilly.com/library/view/learning-spark-2nd/9781492050032/ch09.html)

 _pyspark_ 

In [2]:
# In Python
# Configure source data path

# In Python
import os
from pyspark.sql import SparkSession
# Create a SparkSession


#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 pyspark-shell'

PARENT_DIR = os.popen('dirname $PWD').read().strip()

spark = SparkSession.builder.appName("MyApp") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

sourcePath = os.path.join(
    PARENT_DIR,
    'databricks-datasets/learning-spark-v2/loans/'+\
              'loan-risks.snappy.parquet'
)

# Configure Delta Lake path
deltaPath = os.path.join("/tmp","loans_delta")

# Create the Delta Lake table with the same loans data
(spark.read.format("parquet")
           .load(sourcePath)
           .write
           .mode('overwrite')
           .format("delta")
           .save(deltaPath))

# Create a view on the data called loans_delta
(spark.read.format("delta")
 .load(deltaPath)
 .createOrReplaceTempView("loans_delta"))

In [3]:
filePath = """/databricks-datasets/learning-spark-v2/sf-airbnb/
sf-airbnb-clean.parquet/"""
airbnbDF = spark.read.parquet(filePath)
airbnbDF.select("neighbourhood_cleansed", "room_type", "bedrooms", "bathrooms", 
                "number_of_reviews", "price").show(5)

+--------+
|count(1)|
+--------+
|   14705|
+--------+



In [4]:
spark.sql("SELECT * FROM loans_delta LIMIT 5").show()

+-------+-----------+---------+----------+
|loan_id|funded_amnt|paid_amnt|addr_state|
+-------+-----------+---------+----------+
|      0|       1000|   182.22|        CA|
|      1|       1000|   361.19|        WA|
|      2|       1000|   176.26|        TX|
|      3|       1000|   1000.0|        OK|
|      4|       1000|   249.98|        PA|
+-------+-----------+---------+----------+



In [4]:
from shutil import rmtree
assert os.path.isdir(deltaPath)
rmtree(deltaPath,True)